In [1]:
import numpy as np
import roboticstoolbox as rtb
from roboticstoolbox import DHRobot
import spatialmath as sm

In [2]:
# Extiendo la clase DHRobot para que incluya el generador de trayectorias joint y cartesiano
class DHRobotGT(DHRobot):
    # Defino algunas variables de la clase, que tienen sentido cuando se generan trayectorias
    t_ref=[]; q_ref=[]; qd_ref=[]; qdd_ref=[]; tau=[];

    def get_control_reference(self,t):
        if len(self.t_ref) == 0:
            raise ValueError("No hay trayectorias deseadas definidas")

        # busca el índice dentro del vector
        k = np.searchsorted(self.t_ref, t, side='right') - 1

        # Dejo al índice dentro del rango
        if k < 0:
            k = 0
        elif k >= len(self.t_ref):
            k = len(self.t_ref) - 1

        return self.t_ref[k],self.q_ref[k],self.qd_ref[k],self.qdd_ref[k]

    def __init__(self, *args, tacc=0.1, Ts=1E-3,vmax=np.array([2*np.pi,2*np.pi]), **kwargs):
        super().__init__(*args, **kwargs)
        self.tacc = tacc
        self.Ts = Ts
        self.vmax = vmax

    def interpoladorTrapezoidal(self,A,B,C,Tj):
        """
        Interpolador trapezoidal en zona 1 y 2

        Args
        ----
          A: punto en el que estoy
          B: punto al que estaba yendo en el segmento anterior
          C: punto al que voy
          Tj: tiempo en que se realiza el movimiento

        Returns
        --------
          q_aux: vector interpolado de posiciones
          qd_aux: vector interpolado de velocidades
          qdd_aux: vector interpolado de aceleraciones
        """
        DA = A-B
        DC = C-B

        # Zona 1
        # Generar el vector tseg para Zona 1
        tseg = np.arange(-self.tacc + self.Ts, self.tacc + self.Ts, self.Ts)

        # Calculo las referencias para zona 1
        qdd_aux = np.outer((DC/Tj+DA/self.tacc)/(2*self.tacc),np.ones(len(tseg)))
        qd_aux = (DC / Tj)[:, np.newaxis] * (tseg + self.tacc) / (2 * self.tacc) + (DA / self.tacc)[:, np.newaxis] * (tseg - self.tacc) / (2 * self.tacc)
        q_aux = (DC / Tj)[:, np.newaxis] * (tseg + self.tacc)**2 / (4 * self.tacc) + (DA / self.tacc)[:, np.newaxis] * (tseg - self.tacc)**2 / (4 * self.tacc) + np.outer(B,np.ones(len(tseg)))

        # Zona 2
        # Generar el vector tseg para Zona 2
        tseg = np.arange(self.tacc + self.Ts, Tj - self.tacc + 0.5* self.Ts , self.Ts)

        # Inicializar las matrices theta2p, thetap y theta
        qdd_aux = np.hstack([qdd_aux,np.zeros((len(B), len(tseg)))])   # Suponiendo que B es un vector
        qd_aux = np.hstack([qd_aux,np.outer(DC / Tj, np.ones(len(tseg)))])
        q_aux = np.hstack([q_aux,np.outer(DC / Tj, tseg) +  np.outer(B,np.ones(len(tseg)))])
        return q_aux,qd_aux,qdd_aux

    def genTrJoint(self, q_dest,Td):
        """
        Genera la trayectoria joint para un conjunto de puntos de paso

        Args
        -----
          q_dest: Matriz con los puntos de paso. Cada fila corresponde a un punto
          Td: tiempos deseados de cada movimiento

        Returns
        -------
          t: Vector de tiempo de referencia
          q: Vector de posiciones articulares de referencia
          qd: Vector de velocidades articulares de referencia
          qdd: Vector de aceleraciones articulares de referencia
          POSES: Vector de posiciones cartesianas de referencia
        """
        q = np.empty((self.nlinks,0)); qd = np.empty((self.nlinks,0)); qdd = np.empty((self.nlinks,0))
        A = q_dest[0,:];
        for i in range(len(q_dest)):
          B = q_dest[i,:]
          if i<len(q_dest)-1:
            C = q_dest[i+1,:]
          else:
            C = B
            Td[i] = 0
          Tj = np.max((np.max(np.abs(C-B)/self.vmax),Td[i],2*self.tacc))
          q_aux,qd_aux,qdd_aux = self.interpoladorTrapezoidal(A,B,C,Tj)
          q = np.hstack([q,q_aux]); qd = np.hstack([qd,qd_aux]); qdd = np.hstack([qdd,qdd_aux]);
          A = q[:,-1]
        t = np.linspace(0, q.shape[1],num=q.shape[1])*self.Ts

        # Calculo la trayectoria cartesiana deseada
        POSES = self.fkine(q.transpose()) # .extend([self.fkine(q[:,i]) for i in range(q.shape[1])])
        self.t_ref = t; self.q_ref=q.T; self.qd_ref=qd.T; self.qdd_ref=qdd.T
        self.tau = np.zeros_like(self.q_ref)
        return self.t_ref, self.q_ref, self.qd_ref, self.qdd_ref

    def genTrCart(self,POSE_dest,Td):
        """
        Genera la trayectoria cartesiana para un conjunto de puntos de paso

        Args
        ----
          POSE_dest: Lista con las POSES de paso
          Td: tiempos deseados de cada movimiento

        Returns
        -------
          t: Vector de tiempo de referencia
          q: Vector de posiciones articulares de referencia
          qd: Vector de velocidades articulares de referencia
          qdd: Vector de aceleraciones articulares de referencia
          POSES: Vector de posiciones cartesianas de referencia
        """

        POSEA = POSE_dest[0]
        POSES = []
        for i in range(len(POSE_dest)):
          POSEB = POSE_dest[i]
          if i<len(POSE_dest)-1:
            POSEC = POSE_dest[i+1]
          else:
            POSEC = POSEB
            Td[i] = 0
          A = np.concatenate((POSEA.t,POSEA.eulervec()))
          B = np.concatenate((POSEB.t,POSEB.eulervec()))
          C = np.concatenate((POSEC.t,POSEC.eulervec()))
          Tj = np.max([Td[i],2*self.tacc])

          pos,_,_ = self.interpoladorTrapezoidal(A,B,C,Tj)
          POSES.extend([sm.SE3(pos[0:3,j])*sm.SE3.EulerVec(pos[3:,i]) for j in range(pos.shape[1])])

          POSEA = POSES[-1]

        q = np.zeros((len(POSES),self.nlinks))
        for i in range(len(POSES)):
          q[i,:],_ = self.ikine_a(POSES[i])

        # Obtengo la velocidad articular derivando numéricamente
        qd = np.diff(q, axis=0) / self.Ts
        # Ajustar la longitud de qd para que coincida con q
        qd = np.vstack([qd, np.zeros(self.nlinks,)])

        # Obtengo la aceleración articular derivando numéricamente
        qdd = np.diff(qd, axis=0) / self.Ts
        # Ajustar la longitud de qdd para que coincida con qd
        qdd = np.vstack([qdd, np.zeros(self.nlinks,)])

        t = np.linspace(0, len(q),num=len(q))*self.Ts
        self.t_ref = t; self.q_ref=q; self.qd_ref=qd; self.qdd_ref=qdd
        self.tau = np.zeros_like(self.q_ref)
        return self.t_ref, self.q_ref, self.qd_ref, self.qdd_ref

In [3]:
class myCobot320(DHRobotGT):
    def __init__(self,*args,**kwargs):
        # Definición de los enlaces usando parámetros DH

        eje1 = rtb.RevoluteDH(alpha=-np.pi/2,a=0,d=173.9,offset=0,qlim=[-170*np.pi/180,170*np.pi/180])
        eje2 = rtb.RevoluteDH(alpha=0,a=135,d=0,offset=-np.pi/2,qlim=[-120*np.pi/180,120*np.pi/180])
        eje3 = rtb.RevoluteDH(alpha=0,a=120,d=0,offset=0,qlim=[-148*np.pi/180,148*np.pi/180]) 
        eje4 = rtb.RevoluteDH(alpha=np.pi/2,a=0,d=88.78,offset=np.pi/2,qlim=[-120*np.pi/180,135*np.pi/180])
        eje5 = rtb.RevoluteDH(alpha=-np.pi/2,a=0,d=95,offset=0,qlim=[-169*np.pi/180,169*np.pi/180])    
        eje6 = rtb.RevoluteDH(alpha=0,a=0,d=65.5,offset=0,qlim=[-180*np.pi/180,180*np.pi/180])
    
        # Crear la estructura del robot
        super().__init__(*args,[eje1, eje2, eje3, eje4, eje5, eje6], name='myCobot320',gravity = np.array([0, 0, -9.8]),**kwargs)
    
    def ikine(self,POSE,conf=np.array([1,1,1])):
        """
        Cinemática Inversa del robot myCobot320

        Parameters
        ----------
        POSE : pose de destino. POSE.R corresponde a la rotación (3x3) y POSE.t a la traslación (3x1)
        conf : vector de configuraciones (brazo, codo, muñeca)

        Returns
        -------
        q : vector de variables articulares
        """
        conf1, conf2, conf3 = conf

        # Adecuo las variables POSE 
        px, py, pz = POSE.t
        nx, ny, nz = POSE.R[:,0]
        sx, sy, sz = POSE.R[:,1]
        ax, ay, az = POSE.R[:,2]

        # Tomo los largos de eslabones de la tabla DH
        d1 = self.links[0].d
        a2 = self.links[1].a
        a3 = self.links[2].a
        d4 = self.links[3].d 
        d5 = self.links[4].d 
        d6 = self.links[5].d 

        # Theta1: indeterminación del brazo
        discr = (px - ax*d6)**2 + (py - ay*d6)**2 - d4**2
        if discr<0:
            return [],-1
        # q1 = np.arctan2(d4, conf1*np.sqrt(discr)) - np.arctan2(py - ay*d6, ax*d6 - px)
        q1 = -2*np.arctan((-ax*d6 + px + conf1*np.sqrt(discr))/(-ay*d6 + d4 + py))

        # Theta5: indeterminación de la muñeca
        q5 = np.arctan2(conf3*np.sqrt((ny*np.cos(q1) - nx*np.sin(q1))**2 + (sy*np.cos(q1) - sx*np.sin(q1))**2), ay*np.cos(q1) - ax*np.sin(q1))
        
        # Theta6: singularidad si sin(q5)=0
        # q6 = np.arctan2(-conf3*(sy*np.cos(q1) - sx*np.sin(q1)), conf3*(ny*np.cos(q1) - nx*np.sin(q1)))
        q6 = np.arctan2((sx*np.sin(q1) - sy*np.cos(q1))/np.sin(q5), (ny*np.cos(q1) - nx*np.sin(q1))/np.sin(q5))
        
        # Calculo theta2+theta3+theta4
        # theta234 = np.arctan2(az*conf3, -conf3*(ax*np.cos(q1) + ay*np.sin(q1)))
        theta234 = np.arctan2(az/np.sin(q5), -(ax*np.cos(q1) + ay*np.sin(q1))/np.sin(q5))

        # Cálculos auxiliares: sustituciones A y B
        A = px*np.cos(q1) - d5*np.sin(theta234) + py*np.sin(q1) + d6*np.sin(q5)*np.cos(theta234)
        B = d1 - pz + d5*np.cos(theta234) + d6*np.sin(q5)*np.sin(theta234)
        # Calculo c3 
        c3 = (A**2 + B**2 - a2**2 - a3**2)/(2*a2*a3)
        # Chequeo que el punto sea alcanzable
        if np.abs(c3)>1:
            return [],-1
        # Theta3: indeterminación del codo 
        q3 = np.arctan2(conf2*np.sqrt(1 - c3**2), c3)
        
        # Calculo el seno y el coseno de theta2
        s2 = (B*a2 + B*a3*np.cos(q3) - A*a3*np.sin(q3))/(a2**2 + a3**2 + 2*a2*a3*np.cos(q3))
        c2 = (A*a2 + A*a3*np.cos(q3) + B*a3*np.sin(q3))/(a2**2 + a3**2 + 2*a2*a3*np.cos(q3))
        # Calculo theta2
        q2 = np.arctan2(s2, c2)
        
        # Calculo theta4
        q4 = theta234 - q2 - q3
        
        q = np.array([q1,q2,q3,q4,q5,q6]) #- self.offset
        # Limito q entre -pi y pi
        q = (q + np.pi) % (2 * np.pi) - np.pi
        status=1
        return q,status

In [4]:
cobot = myCobot320()

# Cálculo de configuraciones

Dado un vector con 6 variables articulares aleatorias, buscamos las 3 configuraciones que corresponden a la pose. Para esto calculamos primero la pose con el PCD y luego aplicamos el PCI para hallar las variables articulares y ver si coinciden con las aleatorias. Se pueden iterar las 8 formas posibles de llegar a la pose y dar con la correcta o simplificar el cálculo de forma que:

$$\mathrm{conf_1} =- \mathrm{sign} \: {p^6_1}_x \\
\mathrm{conf_2} = \mathrm{sign} \: q_3 \\
\mathrm{conf_3} = \mathrm{sign} \: q_5$$

Con el siguiente código iteramos 10000 veces para ver si efectivamente estas igualdades se cumplen. La comparación se hace con el método que itera una a una las configuraciones posibles hasta dar con la correcta. Como código comentado se incluye la pose y las variables articulares. Podría adaptarse a una función que calcule las configuraciones si la aproximación recién mencionada no llegase a ser suficiente.

In [5]:
import itertools

num_pruebas = 10000
tolerancia = 1e-9

# Genera las 8 posibles combinaciones de configuraciones
combinations = list(itertools.product([1, -1], repeat=3))
chequeo_conf1 = 0
chequeo_conf2 = 0
chequeo_conf3 = 0
no_encontrados = 0

for prueba in range(num_pruebas):
    q = np.random.randn(6)
    pose = cobot.fkine(q)
    
    # Flag para chequear si se encontró una solución válida
    sol_encontr = False

    for comb in combinations:
        q_ikine = cobot.ikine(pose, np.array(comb))[0]

        # Chequeamos que el pci se haya resuelto correctamente
        if len(q_ikine) > 0:
            # Consideramos el offset de los ejes 2 y 4
            q_ikine[1] = q_ikine[1]+np.pi/2
            q_ikine[3] = q_ikine[3]-np.pi/2

            # Comparamos q con la calculada con ikine usando una tolerancia
            if np.allclose(q[[0, 2, 4, 5]], q_ikine[[0, 2, 4, 5]], atol=tolerancia): 
                A = cobot.fkine_all(q)
                p16_x = (np.linalg.inv(A[1]) @ A[6].A)[0, -1]
                # print(f"Q rand = {q}")
                # print(f"Q ikine = {q_ikine}")
                # print(f'conf = {comb}')
                # print(f'Diferencia q2 = {(q[1] - q_ikine[1])/np.pi:.2f}*pi')
                # print(f'Diferencia q4 = {(q[3] - q_ikine[3])/np.pi:.2f}*pi')
                # print(pose)
                # print(f'p16_x = {p16_x}')
                # print(f'sign p16_x = -conf1? {np.sign(p16_x)==-comb[0]}')
                if np.sign(p16_x)==-comb[0]: chequeo_conf1+=1
                # if np.sign(q[0])==comb[0]: chequeo_conf1+=1
                if np.sign(q[2])==comb[1]: chequeo_conf2+=1
                if np.sign(q[4])==comb[2]: chequeo_conf3+=1
                sol_encontr = True
                break  # Si se encontró la configuración pasamos a la próxima prueba
            continue
    
    # Si los q aleatorios implican una pose fuera de alcance:
    if not sol_encontr:
        # print(f"Prueba {prueba+1}: no se encontró una solución válida para q = {q}.")
        no_encontrados += 1
        
    # print("-" * 50)
print(f'Chequeo de conf1 = -sign(p16_x): {100*chequeo_conf1/(num_pruebas-no_encontrados)} %')
print(f'Chequeo de conf2 = sign(q3): {100*chequeo_conf2/(num_pruebas-no_encontrados)} %')
print(f'Chequeo de conf3 = sign(q5): {100*chequeo_conf3/(num_pruebas-no_encontrados)} %')

Chequeo de conf1 = -sign(p16_x): 96.43540428959822 %
Chequeo de conf2 = sign(q3): 100.0 %
Chequeo de conf3 = sign(q5): 100.0 %


Se ve que hay una coincidencia de un $96 \, \%$ aproximadamente para $\mathrm{conf}_1$. Esta aproximación puede considerarse válida. Queda pendiente analizar aquellas poses donde esto no ocurre.

Respecto a los casos donde no se encuentra la solución, ocurre que la función utilizada para generar las variables articulares aleatorias a veces da valores que exceden en módulo el valor de $\pi$. Al estar limitado el PCI a valores entre $\pi$ y $-\pi$, claramente no habrá coincidencia.

Otra cuestión interesante surge cuando es la corrección por el offset en $q_2$ y $q_4$ la que hace que los mismos excedan el intervalo $\left[\pi \, , \,-\pi\right]$.

A continuación 2 vectores de variables articulares a modo de ejemplo:

In [6]:
qq1 = [0.09724027, -1.96455596,  1.35916914, -1.40823184, -1.11264853,  2.71145273]
qq2 = [-0.77592659,  0.11917341,  0.27168502,  1.8692549,   2.21533094, -0.56688381]

In [7]:
test = np.array(cobot.ikine(cobot.fkine(qq1), np.array([1, 1, -1]))[0])
test[1] = test[1] + np.pi/2
test[3] = test[3] - np.pi/2
print(test)
print(f'q2 - q2_ikine = {(test[1] - qq1[1])/np.pi}*pi')

[ 0.09724027  4.31862935  1.35916914 -1.40823184 -1.11264853  2.71145273]
q2 - q2_ikine = 2.0*pi


In [8]:
test2 = np.array(cobot.ikine(cobot.fkine(qq2), np.array([-1, 1, 1]))[0])
test2[1] = test2[1] + np.pi/2
test2[3] =test2[3] - np.pi/2
print(test2)
print(f'q4 - q4_ikine = {(test2[3] - qq2[3])/np.pi}*pi')

[-0.77592659  0.11917341  0.27168502 -4.41393041  2.21533094 -0.56688381]
q4 - q4_ikine = -2.0*pi


Los valores para las variables articulares son múltiplos de $2 \, \pi$. De esta manera, son equivalentes geométricamente pero podría ser necesario dividirlos por $2 \, \pi$ para mantener acotado el intervalo de trabajo y evitar problemas con los límites de los ejes.